# Assignment 07 : Team 06

#### Rishabh Indoria
#### Shruti Tambe
#### Hitesh Pant


# Problem Statement

To design a chatbot

In [1]:
# Importing necessary Libraries

import csv
import itertools
import operator
import numpy as np
import nltk
import sys
from datetime import datetime
import string

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Download NLTK model data (you need to do this once)
nltk.download("book")

[nltk_data] Downloading collection 'book'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/shrutitambe/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /Users/shrutitambe/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package chat80 to
[nltk_data]    |     /Users/shrutitambe/nltk_data...
[nltk_data]    |   Package chat80 is already up-to-date!
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/shrutitambe/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package conll2000 to
[nltk_data]    |     /Users/shrutitambe/nltk_data...
[nltk_data]    |   Package conll2000 is already up-to-date!
[nltk_data]    | Downloading package conll2002 to
[nltk_data]    |     /Users/shrutitambe/nltk_data...
[nltk_data]    |   Package conll2002 is already up-t

True

Setting up some variables for text processing tasks.

In [3]:
vocabulary_size = 3000 # can be improved by using a larger vocabulary size

unknown_token = "UNKNOWN_TOKEN" # for words not in vocabulary
sentence_start_token = "START" # representing start of a sentence
sentence_end_token = 'END' # representing end of a sentence

# Preprocessing

Using indicnlp library to do natural language processing on Indian languages. In our case, we are using Hindi. We chose indicnlp because it has better tools and resources for working with Indian languages, such as tokenization and transliteration.

In [4]:
!pip install indic-nlp-library

In [5]:
import indicnlp

Splitting a Hindi text file into sentences using the indicnlp library.

In [6]:
from indicnlp.tokenize import sentence_tokenize, indic_tokenize
import string
with open(r"thoughts", 'r', encoding="utf-8") as f:
    content = f.read()
# Tokenize the conversation into sentences
text = content.strip()
text = text.replace("\n"," ")
text = ''.join([char for char in text if char not in string.punctuation and not char.isdigit()])
sentences = sentence_tokenize.sentence_split(text, lang='hi')

The text we took as our dataset look someting like this :

In [7]:
text

'अपने भीतर मुड़ना ही एकमात्र समाधान हैसद्गुरु की नवीनतम पुस्तक ‘इनर इंजीनरिंग’ से लिए गए इस अंश में सद्गुरु हमें स्थायी रूप से खुशहाली का अनुभव करने के एकमात्र तरीके से परिचित कराते हैं। सद्गुरु आपने अपनी जिंदगी में अब तक हर चीज बस एक चीज की तलाश में की है। चाहे आपने बेहतर करियर खोजा हो कारोबार शुरू किया हो पैसे कमाए हों या फिर परिवार को पालापोसा हो यह सब आपने इसलिए किया क्योंकि आप बस एक साधारण सी चीज चाहते थेः खुशी। लेकिन रास्ते में कहीं न कहीं जीवन उलझ गया।   अगर आप इस धरती पर किसी दूसरे जीव के रूप में पैदा हुए होते तो वह बहुत सरल होता। आपकी जरूरतें सिर्फ शारीरिक होतीं। पेट भर खाना मिल जाए तो वह एक बहुत शानदार दिन कहलाता। अपने कुत्ते या अपनी बिल्ली पर गौर कीजिएः जिस पल उनका पेट भर जाता है वे काफी शांतिमय होते हैं।   लेकिन जब आप इस दुनिया में एक इंसान के रूप में आते हैं तो चीजें बदल जाती हैं। खाली पेट की एक ही समस्या हैः भूख। लेकिन अगर पेट भरा हुआ हो तो सैकड़ों समस्याएं हैं जब सवाल हमारे जीवनसंरक्षण का होता है तो वह हमारे जीवन का एक बड़ा मुद्दा होता है। लेकिन जिस पल इस मुद्दे का ख्याल रख

This code will make a list of sentences, each with start and end tokens added. This is a helpful step in creating new text.

In [8]:
append_start_token = [sentence_start_token + ' ' + x + ' ' + sentence_end_token for x in sentences ]

In [9]:
append_start_token

['START अपने भीतर मुड़ना ही एकमात्र समाधान हैसद्गुरु की नवीनतम पुस्तक ‘इनर इंजीनरिंग’ से लिए गए इस अंश में सद्गुरु हमें स्थायी रूप से खुशहाली का अनुभव करने के एकमात्र तरीके से परिचित कराते हैं। END',
 'START सद्गुरु आपने अपनी जिंदगी में अब तक हर चीज बस एक चीज की तलाश में की है। END',
 'START चाहे आपने बेहतर करियर खोजा हो कारोबार शुरू किया हो पैसे कमाए हों या फिर परिवार को पालापोसा हो यह सब आपने इसलिए किया क्योंकि आप बस एक साधारण सी चीज चाहते थेः खुशी। END',
 'START लेकिन रास्ते में कहीं न कहीं जीवन उलझ गया। END',
 'START अगर आप इस धरती पर किसी दूसरे जीव के रूप में पैदा हुए होते तो वह बहुत सरल होता। END',
 'START आपकी जरूरतें सिर्फ शारीरिक होतीं। END',
 'START पेट भर खाना मिल जाए तो वह एक बहुत शानदार दिन कहलाता। END',
 'START अपने कुत्ते या अपनी बिल्ली पर गौर कीजिएः जिस पल उनका पेट भर जाता है वे काफी शांतिमय होते हैं। END',
 'START लेकिन जब आप इस दुनिया में एक इंसान के रूप में आते हैं तो चीजें बदल जाती हैं। END',
 'START खाली पेट की एक ही समस्या हैः भूख। END',
 'START लेकिन अगर पेट भरा ह

This code will tokenize each sentence in the `append_start_token` list into words and print the resulting word tokens for each sentence. This is a common step in natural language processing for breaking down text into its constituent words.

In [10]:
# Tokenize each sentence into words
tokenized_conversation = [indic_tokenize.trivial_tokenize(sentence) for sentence in append_start_token]

# Print the tokenized conversation
for sentence in tokenized_conversation:
    print(sentence)

['START', 'अपने', 'भीतर', 'मुड़ना', 'ही', 'एकमात्र', 'समाधान', 'हैसद्गुरु', 'की', 'नवीनतम', 'पुस्तक', '‘इनर', 'इंजीनरिंग’', 'से', 'लिए', 'गए', 'इस', 'अंश', 'में', 'सद्गुरु', 'हमें', 'स्थायी', 'रूप', 'से', 'खुशहाली', 'का', 'अनुभव', 'करने', 'के', 'एकमात्र', 'तरीके', 'से', 'परिचित', 'कराते', 'हैं', '।', 'END']
['START', 'सद्गुरु', 'आपने', 'अपनी', 'जिंदगी', 'में', 'अब', 'तक', 'हर', 'चीज', 'बस', 'एक', 'चीज', 'की', 'तलाश', 'में', 'की', 'है', '।', 'END']
['START', 'चाहे', 'आपने', 'बेहतर', 'करियर', 'खोजा', 'हो', 'कारोबार', 'शुरू', 'किया', 'हो', 'पैसे', 'कमाए', 'हों', 'या', 'फिर', 'परिवार', 'को', 'पालापोसा', 'हो', 'यह', 'सब', 'आपने', 'इसलिए', 'किया', 'क्योंकि', 'आप', 'बस', 'एक', 'साधारण', 'सी', 'चीज', 'चाहते', 'थेः', 'खुशी', '।', 'END']
['START', 'लेकिन', 'रास्ते', 'में', 'कहीं', 'न', 'कहीं', 'जीवन', 'उलझ', 'गया', '।', 'END']
['START', 'अगर', 'आप', 'इस', 'धरती', 'पर', 'किसी', 'दूसरे', 'जीव', 'के', 'रूप', 'में', 'पैदा', 'हुए', 'होते', 'तो', 'वह', 'बहुत', 'सरल', 'होता', '।', 'END']
['START', 'आपकी

This code counts the number of different words in the text data. The result is printed as "Found X unique words," where X is the number of different words.

In [11]:
# Count the word frequencies
from collections import Counter
word_freq = Counter(word for sent in tokenized_conversation for word in sent)
print("Found %d unique word tokens." % len(word_freq))

Found 720 unique word tokens.


This code is building index-to-word and word-to-index mappings, as we are preparing the most common words.

In [12]:
# Get the most common words and build index_to_word and word_to_index vectors
vocabulary_size = len(word_freq)
index_to_word = list(word_freq.keys())
index_to_word.append(unknown_token)
word_to_index = {w: i for i, w in enumerate(index_to_word)}
print("The least frequent word in our vocabulary is '%s' and appeared %d times." % word_freq.most_common()[-1])

The least frequent word in our vocabulary is 'लाना' and appeared 1 times.


This code goes over each sentence word by word and replaces words that are not in the vocabulary with the special word "unknown_token". It then prints the number of words in the vocabulary and shows an example sentence before and after this step. This makes sure that all words in the text are included in our vocabulary.

In [13]:
# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_conversation):
    tokenized_conversation[i] = [w if w in word_freq else unknown_token for w in sent]

print("Using vocabulary size %d." % vocabulary_size)

print("\nExample sentence: '%s'" % sentences[0])
print("\nExample sentence after pre-processing: '%s'" % tokenized_conversation[0])

Using vocabulary size 720.

Example sentence: 'अपने भीतर मुड़ना ही एकमात्र समाधान हैसद्गुरु की नवीनतम पुस्तक ‘इनर इंजीनरिंग’ से लिए गए इस अंश में सद्गुरु हमें स्थायी रूप से खुशहाली का अनुभव करने के एकमात्र तरीके से परिचित कराते हैं।'

Example sentence after pre-processing: '['START', 'अपने', 'भीतर', 'मुड़ना', 'ही', 'एकमात्र', 'समाधान', 'हैसद्गुरु', 'की', 'नवीनतम', 'पुस्तक', '‘इनर', 'इंजीनरिंग’', 'से', 'लिए', 'गए', 'इस', 'अंश', 'में', 'सद्गुरु', 'हमें', 'स्थायी', 'रूप', 'से', 'खुशहाली', 'का', 'अनुभव', 'करने', 'के', 'एकमात्र', 'तरीके', 'से', 'परिचित', 'कराते', 'हैं', '।', 'END']'


This code will print a list of the most common words in the text data, along with their respective frequencies. Thi step is useful for understanding which words are prevalent in the text and may be important for our task.

In [14]:
# Get the most common words
most_common_words = word_freq.most_common()
print("The most common words:")
for word, frequency in most_common_words:
    print(f"'{word}': {frequency}")

The most common words:
'START': 127
'।': 127
'END': 127
'है': 119
'हैं': 81
'एक': 49
'की': 43
'आप': 43
'से': 40
'के': 40
'में': 38
'और': 36
'तो': 35
'कि': 34
'को': 33
'नहीं': 31
'यह': 27
'वह': 23
'का': 21
'या': 20
'अगर': 20
'पर': 20
'भीतर': 19
'लेकिन': 18
'हम': 18
'लिए': 17
'आपके': 17
'अपने': 16
'कर': 16
'रहे': 16
'ही': 15
'उसे': 15
'इस': 14
'बस': 14
'सकते': 14
'भी': 13
'चीज': 12
'हो': 12
'क्या': 12
'अनुभव': 11
'किया': 11
'जीवन': 11
'किसी': 11
'आपकी': 11
'जब': 11
'आनंद': 11
'बाहर': 11
'रूप': 10
'तक': 10
'हर': 10
'बहुत': 10
'था': 10
'रहा': 10
'साथ': 10
'प्रसन्नता': 10
'कहते': 10
'फिर': 9
'होता': 9
'दुनिया': 9
'सबसे': 9
'सुखद': 9
'खुशहाली': 8
'अपनी': 8
'दिन': 8
'कोई': 8
'ऐसा': 8
'आपको': 8
'सकता': 8
'करने': 7
'हों': 7
'कभी': 7
'आपका': 7
'आंतरिक': 7
'तरह': 7
'तीतर': 7
'आपने': 6
'खुशी': 6
'गया': 6
'होते': 6
'सिर्फ': 6
'वे': 6
'चीजें': 6
'पहुंच': 6
'बाहरी': 6
'ठीक': 6
'करते': 6
'बन': 6
'ने': 6
'साधारण': 5
'चाहते': 5
'धरती': 5
'जाता': 5
'हमारे': 5
'आज': 5
'बिना': 5
'काम': 5
'मानव': 5
'जो': 5


In [15]:
append_start_token[:5]

['START अपने भीतर मुड़ना ही एकमात्र समाधान हैसद्गुरु की नवीनतम पुस्तक ‘इनर इंजीनरिंग’ से लिए गए इस अंश में सद्गुरु हमें स्थायी रूप से खुशहाली का अनुभव करने के एकमात्र तरीके से परिचित कराते हैं। END',
 'START सद्गुरु आपने अपनी जिंदगी में अब तक हर चीज बस एक चीज की तलाश में की है। END',
 'START चाहे आपने बेहतर करियर खोजा हो कारोबार शुरू किया हो पैसे कमाए हों या फिर परिवार को पालापोसा हो यह सब आपने इसलिए किया क्योंकि आप बस एक साधारण सी चीज चाहते थेः खुशी। END',
 'START लेकिन रास्ते में कहीं न कहीं जीवन उलझ गया। END',
 'START अगर आप इस धरती पर किसी दूसरे जीव के रूप में पैदा हुए होते तो वह बहुत सरल होता। END']

## Ngrams:

We use the Counter class from Python to count the number of times pairs of words (bigrams), triples of words (trigrams), and so on appear in the text. We also use the %time magic command to measure how long the code takes to run.

The code prints the 10 most common pairs, triples, and so on, along with their counts. This is useful for finding patterns and relationships between words in the text.

In [16]:
%%time
from collections import Counter
from nltk import ngrams
bigram_counts = Counter(ngrams(text.split(), 2))
bigram_counts.most_common(10)

CPU times: user 2.56 ms, sys: 134 µs, total: 2.69 ms
Wall time: 2.73 ms


[(('है', 'कि'), 17),
 (('के', 'लिए'), 14),
 (('है', 'तो'), 11),
 (('सकते', 'हैं'), 9),
 (('हम', 'उसे'), 8),
 (('कहते', 'हैं।'), 8),
 (('हैं।', 'अगर'), 8),
 (('हैं', 'कि'), 8),
 (('तो', 'हम'), 7),
 (('रहे', 'हैं'), 7)]

In [17]:
%%time
import collections
def ngrams(text, n=2):
    return zip(*[text[i:] for i in range(n)])
bigram_counts = collections.Counter(ngrams(text.split(), 2))
bigram_counts.most_common(10)

CPU times: user 1.87 ms, sys: 208 µs, total: 2.08 ms
Wall time: 2.09 ms


[(('है', 'कि'), 17),
 (('के', 'लिए'), 14),
 (('है', 'तो'), 11),
 (('सकते', 'हैं'), 9),
 (('हम', 'उसे'), 8),
 (('कहते', 'हैं।'), 8),
 (('हैं।', 'अगर'), 8),
 (('हैं', 'कि'), 8),
 (('तो', 'हम'), 7),
 (('रहे', 'हैं'), 7)]

In [18]:
trigram_counts = Counter(ngrams(text.split(), 3))
trigram_counts.most_common(10)

[(('तो', 'हम', 'उसे'), 7),
 (('है', 'तो', 'हम'), 6),
 (('कहते', 'हैं।', 'अगर'), 6),
 (('यह', 'है', 'कि'), 5),
 (('के', 'रूप', 'में'), 4),
 (('बहुत', 'सुखद', 'बन'), 4),
 (('है', 'कि', 'आप'), 4),
 (('इस', 'धरती', 'पर'), 3),
 (('कर', 'सकते', 'हैं'), 3),
 (('करने', 'के', 'लिए'), 3)]

In [19]:
quadgram_counts = Counter(ngrams(text.replace('&', 'i').split(), 4))
quadgram_counts.most_common(10)

[(('है', 'तो', 'हम', 'उसे'), 6),
 (('अगर', 'वह', 'बहुत', 'सुखद'), 3),
 (('वह', 'बहुत', 'सुखद', 'बन'), 3),
 (('सुखद', 'बन', 'जाता', 'है'), 3),
 (('बन', 'जाता', 'है', 'तो'), 3),
 (('की', 'कोशिश', 'कर', 'रहे'), 3),
 (('बाहर', 'निकलने', 'का', 'रास्ता'), 3),
 (('हैं।', 'लेकिन', 'जब', 'आप'), 2),
 (('हैं।', 'ऐसा', 'क्यों', 'है'), 2),
 (('कहते', 'हैं।', 'अगर', 'वह'), 2)]

In [20]:
pentagram_counts = Counter(ngrams(text.split(), 5))
pentagram_counts.most_common(10)

[(('अगर', 'वह', 'बहुत', 'सुखद', 'बन'), 3),
 (('सुखद', 'बन', 'जाता', 'है', 'तो'), 3),
 (('कहते', 'हैं।', 'अगर', 'वह', 'बहुत'), 2),
 (('हैं।', 'अगर', 'वह', 'बहुत', 'सुखद'), 2),
 (('वह', 'बहुत', 'सुखद', 'बन', 'जाता'), 2),
 (('बहुत', 'सुखद', 'बन', 'जाता', 'है'), 2),
 (('बन', 'जाता', 'है', 'तो', 'हम'), 2),
 (('जाता', 'है', 'तो', 'हम', 'उसे'), 2),
 (('की', 'कोशिश', 'कर', 'रहे', 'हों'), 2),
 (('हैं', 'लेकिन', 'समस्या', 'यह', 'है'), 2)]

In [21]:
hexagram_counts = Counter(ngrams(text.split(), 6))
hexagram_counts.most_common(10)

[(('कहते', 'हैं।', 'अगर', 'वह', 'बहुत', 'सुखद'), 2),
 (('हैं।', 'अगर', 'वह', 'बहुत', 'सुखद', 'बन'), 2),
 (('अगर', 'वह', 'बहुत', 'सुखद', 'बन', 'जाता'), 2),
 (('वह', 'बहुत', 'सुखद', 'बन', 'जाता', 'है'), 2),
 (('बहुत', 'सुखद', 'बन', 'जाता', 'है', 'तो'), 2),
 (('सुखद', 'बन', 'जाता', 'है', 'तो', 'हम'), 2),
 (('बन', 'जाता', 'है', 'तो', 'हम', 'उसे'), 2),
 (('हैं', 'लेकिन', 'समस्या', 'यह', 'है', 'कि'), 2),
 (('अपने', 'भीतर', 'मुड़ना', 'ही', 'एकमात्र', 'समाधान'), 1),
 (('भीतर', 'मुड़ना', 'ही', 'एकमात्र', 'समाधान', 'हैसद्गुरु'), 1)]

In [22]:
heptagram_counts = Counter(ngrams(text.split(), 7))
heptagram_counts.most_common(10)

[(('कहते', 'हैं।', 'अगर', 'वह', 'बहुत', 'सुखद', 'बन'), 2),
 (('हैं।', 'अगर', 'वह', 'बहुत', 'सुखद', 'बन', 'जाता'), 2),
 (('अगर', 'वह', 'बहुत', 'सुखद', 'बन', 'जाता', 'है'), 2),
 (('वह', 'बहुत', 'सुखद', 'बन', 'जाता', 'है', 'तो'), 2),
 (('बहुत', 'सुखद', 'बन', 'जाता', 'है', 'तो', 'हम'), 2),
 (('सुखद', 'बन', 'जाता', 'है', 'तो', 'हम', 'उसे'), 2),
 (('अपने', 'भीतर', 'मुड़ना', 'ही', 'एकमात्र', 'समाधान', 'हैसद्गुरु'), 1),
 (('भीतर', 'मुड़ना', 'ही', 'एकमात्र', 'समाधान', 'हैसद्गुरु', 'की'), 1),
 (('मुड़ना', 'ही', 'एकमात्र', 'समाधान', 'हैसद्गुरु', 'की', 'नवीनतम'), 1),
 (('ही', 'एकमात्र', 'समाधान', 'हैसद्गुरु', 'की', 'नवीनतम', 'पुस्तक'), 1)]

In [23]:
octagram_counts = Counter(ngrams(text.split(), 8))
octagram_counts.most_common(10)

[(('कहते', 'हैं।', 'अगर', 'वह', 'बहुत', 'सुखद', 'बन', 'जाता'), 2),
 (('हैं।', 'अगर', 'वह', 'बहुत', 'सुखद', 'बन', 'जाता', 'है'), 2),
 (('अगर', 'वह', 'बहुत', 'सुखद', 'बन', 'जाता', 'है', 'तो'), 2),
 (('वह', 'बहुत', 'सुखद', 'बन', 'जाता', 'है', 'तो', 'हम'), 2),
 (('बहुत', 'सुखद', 'बन', 'जाता', 'है', 'तो', 'हम', 'उसे'), 2),
 (('अपने', 'भीतर', 'मुड़ना', 'ही', 'एकमात्र', 'समाधान', 'हैसद्गुरु', 'की'), 1),
 (('भीतर', 'मुड़ना', 'ही', 'एकमात्र', 'समाधान', 'हैसद्गुरु', 'की', 'नवीनतम'),
  1),
 (('मुड़ना', 'ही', 'एकमात्र', 'समाधान', 'हैसद्गुरु', 'की', 'नवीनतम', 'पुस्तक'),
  1),
 (('ही', 'एकमात्र', 'समाधान', 'हैसद्गुरु', 'की', 'नवीनतम', 'पुस्तक', '‘इनर'),
  1),
 (('एकमात्र',
   'समाधान',
   'हैसद्गुरु',
   'की',
   'नवीनतम',
   'पुस्तक',
   '‘इनर',
   'इंजीनरिंग’'),
  1)]

In [24]:
eneagram_counts = Counter(ngrams(text.split(), 9))
eneagram_counts.most_common(10)

[(('कहते', 'हैं।', 'अगर', 'वह', 'बहुत', 'सुखद', 'बन', 'जाता', 'है'), 2),
 (('हैं।', 'अगर', 'वह', 'बहुत', 'सुखद', 'बन', 'जाता', 'है', 'तो'), 2),
 (('अगर', 'वह', 'बहुत', 'सुखद', 'बन', 'जाता', 'है', 'तो', 'हम'), 2),
 (('वह', 'बहुत', 'सुखद', 'बन', 'जाता', 'है', 'तो', 'हम', 'उसे'), 2),
 (('अपने',
   'भीतर',
   'मुड़ना',
   'ही',
   'एकमात्र',
   'समाधान',
   'हैसद्गुरु',
   'की',
   'नवीनतम'),
  1),
 (('भीतर',
   'मुड़ना',
   'ही',
   'एकमात्र',
   'समाधान',
   'हैसद्गुरु',
   'की',
   'नवीनतम',
   'पुस्तक'),
  1),
 (('मुड़ना',
   'ही',
   'एकमात्र',
   'समाधान',
   'हैसद्गुरु',
   'की',
   'नवीनतम',
   'पुस्तक',
   '‘इनर'),
  1),
 (('ही',
   'एकमात्र',
   'समाधान',
   'हैसद्गुरु',
   'की',
   'नवीनतम',
   'पुस्तक',
   '‘इनर',
   'इंजीनरिंग’'),
  1),
 (('एकमात्र',
   'समाधान',
   'हैसद्गुरु',
   'की',
   'नवीनतम',
   'पुस्तक',
   '‘इनर',
   'इंजीनरिंग’',
   'से'),
  1),
 (('समाधान',
   'हैसद्गुरु',
   'की',
   'नवीनतम',
   'पुस्तक',
   '‘इनर',
   'इंजीनरिंग’',
   'से',
   'लिए'),
  1)]

In [25]:
decagram_counts = Counter(ngrams(text.split(), 10))
decagram_counts.most_common(10)

[(('कहते', 'हैं।', 'अगर', 'वह', 'बहुत', 'सुखद', 'बन', 'जाता', 'है', 'तो'), 2),
 (('हैं।', 'अगर', 'वह', 'बहुत', 'सुखद', 'बन', 'जाता', 'है', 'तो', 'हम'), 2),
 (('अगर', 'वह', 'बहुत', 'सुखद', 'बन', 'जाता', 'है', 'तो', 'हम', 'उसे'), 2),
 (('अपने',
   'भीतर',
   'मुड़ना',
   'ही',
   'एकमात्र',
   'समाधान',
   'हैसद्गुरु',
   'की',
   'नवीनतम',
   'पुस्तक'),
  1),
 (('भीतर',
   'मुड़ना',
   'ही',
   'एकमात्र',
   'समाधान',
   'हैसद्गुरु',
   'की',
   'नवीनतम',
   'पुस्तक',
   '‘इनर'),
  1),
 (('मुड़ना',
   'ही',
   'एकमात्र',
   'समाधान',
   'हैसद्गुरु',
   'की',
   'नवीनतम',
   'पुस्तक',
   '‘इनर',
   'इंजीनरिंग’'),
  1),
 (('ही',
   'एकमात्र',
   'समाधान',
   'हैसद्गुरु',
   'की',
   'नवीनतम',
   'पुस्तक',
   '‘इनर',
   'इंजीनरिंग’',
   'से'),
  1),
 (('एकमात्र',
   'समाधान',
   'हैसद्गुरु',
   'की',
   'नवीनतम',
   'पुस्तक',
   '‘इनर',
   'इंजीनरिंग’',
   'से',
   'लिए'),
  1),
 (('समाधान',
   'हैसद्गुरु',
   'की',
   'नवीनतम',
   'पुस्तक',
   '‘इनर',
   'इंजीनरिंग’',
   'से',
   'लिए',
   '

### Creating the training dataset of corpus

In [26]:
text[0:1000]

'अपने भीतर मुड़ना ही एकमात्र समाधान हैसद्गुरु की नवीनतम पुस्तक ‘इनर इंजीनरिंग’ से लिए गए इस अंश में सद्गुरु हमें स्थायी रूप से खुशहाली का अनुभव करने के एकमात्र तरीके से परिचित कराते हैं। सद्गुरु आपने अपनी जिंदगी में अब तक हर चीज बस एक चीज की तलाश में की है। चाहे आपने बेहतर करियर खोजा हो कारोबार शुरू किया हो पैसे कमाए हों या फिर परिवार को पालापोसा हो यह सब आपने इसलिए किया क्योंकि आप बस एक साधारण सी चीज चाहते थेः खुशी। लेकिन रास्ते में कहीं न कहीं जीवन उलझ गया।   अगर आप इस धरती पर किसी दूसरे जीव के रूप में पैदा हुए होते तो वह बहुत सरल होता। आपकी जरूरतें सिर्फ शारीरिक होतीं। पेट भर खाना मिल जाए तो वह एक बहुत शानदार दिन कहलाता। अपने कुत्ते या अपनी बिल्ली पर गौर कीजिएः जिस पल उनका पेट भर जाता है वे काफी शांतिमय होते हैं।   लेकिन जब आप इस दुनिया में एक इंसान के रूप में आते हैं तो चीजें बदल जाती हैं। खाली पेट की एक ही समस्या हैः भूख। लेकिन अगर पेट भरा हुआ हो तो सैकड़ों समस्याएं हैं जब सवाल हमारे जीवनसंरक्षण का होता है तो वह हमारे जीवन का एक बड़ा मुद्दा होता है। लेकिन जिस पल इस मुद्दे का ख्याल रख

In [27]:
import re

### Extract the first word from each sentence

This code breaks each sentence into words using spaces. Then, it takes the first word of each sentence and puts it in a list called first_words. Later, we will find the most common first word and how often it is used to start a sentence.

In [28]:
# Extract the first word from each sentence
first_words = [sentence.split()[0] for sentence in sentences]

In [29]:
# Extract the first word from each sentence
first_words = [sentence.split()[0] for sentence in sentences]

# Count the frequency of each first word
word_freq = nltk.FreqDist(first_words)

# Get the most common first words
most_common_first_words = word_freq.most_common()
print("The most common first words in sentences:")
for word, frequency in most_common_first_words:
    print(f"'{word}': {frequency}")

The most common first words in sentences:
'अगर': 13
'लेकिन': 7
'आप': 7
'किसी': 4
'एक': 4
'तो': 4
'अपने': 3
'आज': 3
'ऐसा': 3
'जब': 3
'और': 3
'फिर': 3
'वह': 3
'’': 3
'वे': 2
'सबसे': 2
'इसके': 2
'बस': 2
'अब': 2
'सिर्फ': 2
'अभी': 2
'पीड़ा': 2
'सद्गुरु': 1
'चाहे': 1
'आपकी': 1
'पेट': 1
'खाली': 1
'सौ': 1
'हम': 1
'हमने': 1
'खुशहाली': 1
'यह': 1
'दूसरों': 1
'वही': 1
'इंसानइंसान': 1
'नींद': 1
'उसने': 1
'कम': 1
'मानवीय': 1
'परिणाम': 1
'सीधेसीधे': 1
'हमें': 1
'खुद': 1
'बैल': 1
'खेत': 1
'दो': 1
'ये': 1
'पूरी': 1
'कहानी': 1
'मौसम': 1
'स्टॉक': 1
'होने': 1
'आपको': 1
'मानव': 1
'इंसान': 1
'क्या': 1
'हर': 1
'रोशनी': 1
'पूरा': 1
'सारे': 1
'साधारण': 1
'बिना': 1
'इससे': 1
'शांति': 1
'असल': 1
'योग': 1
'आंतरिक': 1
'मैं': 1
'समझ': 1
'क्लेश': 1


# Preparing training data

In this code, we are preparing training data for a sequence-to-sequence model. We are using X_train and y_train to represent input sequences and corresponding target sequences.

In [30]:
#X_train = [[sentence_start_token] for sent,times in first_word_counts if sent != 'o.']
#y_train = [sent for sent in first_word_counts if sent != 'o.']
X_train = [[sentence_start_token] for sent in first_words if sent != 'मीरा।']
y_train = [[sent] for sent in first_words if sent != 'मीरा।']

In [31]:
X_train = [item for sublist in X_train for item in sublist]
y_train = [item for sublist in y_train for item in sublist]

In [32]:
X_train[0:10]

['START',
 'START',
 'START',
 'START',
 'START',
 'START',
 'START',
 'START',
 'START',
 'START']

In [33]:
print(y_train)

['अपने', 'सद्गुरु', 'चाहे', 'लेकिन', 'अगर', 'आपकी', 'पेट', 'अपने', 'लेकिन', 'खाली', 'लेकिन', 'लेकिन', 'किसी', 'आज', 'आप', 'आप', 'सौ', 'हम', 'हमने', 'अगर', 'ऐसा', 'खुशहाली', 'अगर', 'अगर', 'अगर', 'अगर', 'अगर', 'अगर', 'अगर', 'आप', 'जब', 'जब', 'अगर', 'लेकिन', 'वे', 'वे', 'और', 'सबसे', 'यह', 'आज', 'लेकिन', 'दूसरों', 'जब', 'एक', 'इसके', 'तो', 'सबसे', 'ऐसा', 'बस', 'अब', 'वही', 'तो', 'इंसानइंसान', 'तो', 'सिर्फ', 'अपने', 'किसी', 'नींद', 'उसने', 'फिर', 'वह', 'वह', 'फिर', 'कम', 'अभी', 'मानवीय', 'आप', 'परिणाम', 'बस', 'सीधेसीधे', 'किसी', 'लेकिन', 'हमें', 'खुद', 'किसी', 'बैल', 'खेत', 'अब', '’', 'दो', '’', 'ये', 'पूरी', '’', 'और', 'एक', 'वह', 'एक', 'कहानी', 'मौसम', 'स्टॉक', 'और', 'होने', 'फिर', 'आपको', 'मानव', 'पीड़ा', 'इंसान', 'आप', 'अभी', 'आप', 'क्या', 'तो', 'आप', 'हर', 'रोशनी', 'अगर', 'पूरा', 'सारे', 'अगर', 'आज', 'एक', 'साधारण', 'ऐसा', 'बिना', 'इससे', 'शांति', 'असल', 'योग', 'आंतरिक', 'मैं', 'अगर', 'इसके', 'समझ', 'क्लेश', 'पीड़ा', 'सिर्फ']


In [34]:
len(X_train), len(y_train)

(127, 127)

### Shuffling our training data:

This Python function, fisher_yates, randomly mixes two arrays, arr1 and arr2, in the same order. This means that the elements in the two arrays stay paired up after the shuffle. This is useful for randomizing the order of elements while keeping track of which elements go together.

In [35]:
import random

def fisher_yates (arr1, arr2):

    # We will Start from the last element
    # and swap one by one.
    n = len(arr1)
    if n != len(arr2):
        return None

    for i in range(n - 1, 0, -1):

        # Pick a random index from 0 to i
        j = random.randint(0, i)
        #print(i, j)

        # Swap arr[i] with the element at random index
        arr1[i], arr1[j] = arr1[j], arr1[i]
        arr2[i], arr2[j] = arr2[j], arr2[i]

    return arr1, arr2

In [36]:
X_train, y_train = fisher_yates(X_train, y_train)
len(X_train), len(y_train)

(127, 127)

In [37]:
print(y_train)

['आंतरिक', 'यह', 'अगर', 'अगर', 'आज', 'बिना', 'सबसे', 'किसी', 'रोशनी', 'खुद', 'लेकिन', 'सारे', 'सौ', 'कम', 'अगर', 'सद्गुरु', 'ऐसा', 'अगर', 'आप', 'पूरा', 'आज', 'लेकिन', 'इससे', 'अगर', 'खेत', 'वह', 'वे', 'किसी', 'आप', 'दो', 'उसने', 'सबसे', 'अगर', '’', 'अगर', 'पीड़ा', 'परिणाम', 'आज', 'अगर', 'अब', 'और', 'फिर', 'मौसम', '’', 'आप', 'फिर', 'इसके', 'असल', 'लेकिन', 'अगर', 'इंसानइंसान', 'मानवीय', 'लेकिन', 'पीड़ा', 'मानव', 'आप', 'जब', 'फिर', 'लेकिन', 'जब', 'लेकिन', 'अपने', 'वह', 'किसी', 'अगर', 'तो', 'तो', 'एक', 'और', 'नींद', 'एक', 'बैल', 'सिर्फ', 'वे', 'स्टॉक', 'सीधेसीधे', 'पेट', 'बस', 'शांति', 'इंसान', 'आपको', 'क्या', 'योग', 'आप', 'अगर', 'मैं', 'अभी', 'हम', 'एक', 'एक', 'अब', 'आप', 'वह', 'बस', 'और', 'हर', 'इसके', 'तो', 'अपने', '’', 'पूरी', 'क्लेश', 'तो', 'कहानी', 'होने', 'अभी', 'समझ', 'हमें', 'अगर', 'लेकिन', 'दूसरों', 'अपने', 'जब', 'आपकी', 'ऐसा', 'अगर', 'वही', 'ये', 'खुशहाली', 'हमने', 'आप', 'ऐसा', 'चाहे', 'सिर्फ', 'साधारण', 'किसी', 'खाली']


The code below assumes that `X_train` and `y_train` contain words that are present in the `word_to_index` mapping. 
If any word is not found in the mapping, it will be excluded from `X_tokens` and `y_tokens` lists.

In [38]:
X_tokens = [[word_to_index[symbol]] for symbol,word in zip(X_train, y_train) if word in word_to_index]
y_tokens = [[word_to_index[word]] for symbol,word in zip(X_train, y_train) if word in word_to_index]

In [39]:
X_tokens

[[0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0]]

In [40]:
X_train = X_tokens
y_train = y_tokens

In [41]:
len(X_train), len(y_train)

(127, 127)

In [42]:
X_train[0:5], y_train[0:5]

([[0], [0], [0], [0], [0]], [[332], [61], [78], [78], [147]])

## frequency distribution of our ngrams for different values of n.
This code counts how often different groups of words (called n-grams) appear in the text. It does this for different group sizes (n-values). This information can be used to understand how the text is structured and how words are used together.

In [43]:
ngrams_up_to_20 = []
for i in range(2, 21):
    ngram_counts = Counter(ngrams(text.split(), i))
    print('ngram-', i, 'length:', len(ngram_counts))
    ngrams_up_to_20.append(ngram_counts)

ngram- 2 length: 1952
ngram- 3 length: 2309
ngram- 4 length: 2396
ngram- 5 length: 2420
ngram- 6 length: 2424
ngram- 7 length: 2425
ngram- 8 length: 2425
ngram- 9 length: 2425
ngram- 10 length: 2425
ngram- 11 length: 2425
ngram- 12 length: 2425
ngram- 13 length: 2425
ngram- 14 length: 2424
ngram- 15 length: 2423
ngram- 16 length: 2422
ngram- 17 length: 2421
ngram- 18 length: 2420
ngram- 19 length: 2419
ngram- 20 length: 2418


In [44]:
ngrams_up_to_20[0].most_common(10)

[(('है', 'कि'), 17),
 (('के', 'लिए'), 14),
 (('है', 'तो'), 11),
 (('सकते', 'हैं'), 9),
 (('हम', 'उसे'), 8),
 (('कहते', 'हैं।'), 8),
 (('हैं।', 'अगर'), 8),
 (('हैं', 'कि'), 8),
 (('तो', 'हम'), 7),
 (('रहे', 'हैं'), 7)]

In [45]:
ngrams_up_to_20[1].most_common(10)

[(('तो', 'हम', 'उसे'), 7),
 (('है', 'तो', 'हम'), 6),
 (('कहते', 'हैं।', 'अगर'), 6),
 (('यह', 'है', 'कि'), 5),
 (('के', 'रूप', 'में'), 4),
 (('बहुत', 'सुखद', 'बन'), 4),
 (('है', 'कि', 'आप'), 4),
 (('इस', 'धरती', 'पर'), 3),
 (('कर', 'सकते', 'हैं'), 3),
 (('करने', 'के', 'लिए'), 3)]

In [46]:
ngrams_up_to_20[2].most_common(10)

[(('है', 'तो', 'हम', 'उसे'), 6),
 (('अगर', 'वह', 'बहुत', 'सुखद'), 3),
 (('वह', 'बहुत', 'सुखद', 'बन'), 3),
 (('सुखद', 'बन', 'जाता', 'है'), 3),
 (('बन', 'जाता', 'है', 'तो'), 3),
 (('की', 'कोशिश', 'कर', 'रहे'), 3),
 (('बाहर', 'निकलने', 'का', 'रास्ता'), 3),
 (('हैं।', 'लेकिन', 'जब', 'आप'), 2),
 (('हैं।', 'ऐसा', 'क्यों', 'है'), 2),
 (('कहते', 'हैं।', 'अगर', 'वह'), 2)]

In [47]:
bigrams_to_learn = bigram_counts.most_common(100)
bigrams_to_learn[0][0], bigrams_to_learn[0][0][0], bigrams_to_learn[0][0][1]

(('है', 'कि'), 'है', 'कि')

In [48]:
len(ngrams_up_to_20[0].most_common()), ngrams_up_to_20[0].most_common()

(1952,
 [(('है', 'कि'), 17),
  (('के', 'लिए'), 14),
  (('है', 'तो'), 11),
  (('सकते', 'हैं'), 9),
  (('हम', 'उसे'), 8),
  (('कहते', 'हैं।'), 8),
  (('हैं।', 'अगर'), 8),
  (('हैं', 'कि'), 8),
  (('तो', 'हम'), 7),
  (('रहे', 'हैं'), 7),
  (('अपने', 'भीतर'), 6),
  (('हैं', 'तो'), 6),
  (('रहा', 'है'), 6),
  (('के', 'साथ'), 6),
  (('हैं', 'लेकिन'), 6),
  (('कि', 'आप'), 6),
  (('कर', 'रहे'), 6),
  (('रूप', 'से'), 5),
  (('हर', 'चीज'), 5),
  (('जब', 'आप'), 5),
  (('की', 'एक'), 5),
  (('होता', 'है।'), 5),
  (('हैं।', 'आप'), 5),
  (('नहीं', 'है'), 5),
  (('सुखद', 'बन'), 5),
  (('यह', 'है'), 5),
  (('की', 'जरूरत'), 5),
  (('रहे', 'हैं।'), 5),
  (('करने', 'के'), 4),
  (('धरती', 'पर'), 4),
  (('के', 'रूप'), 4),
  (('रूप', 'में'), 4),
  (('तो', 'वह'), 4),
  (('वह', 'बहुत'), 4),
  (('जाता', 'है'), 4),
  (('होते', 'हैं।'), 4),
  (('कर', 'सकते'), 4),
  (('है।', 'अगर'), 4),
  (('बहुत', 'सुखद'), 4),
  (('आनंद', 'या'), 4),
  (('है', 'और'), 4),
  (('तो', 'आप'), 4),
  (('नहीं', 'है।'), 4),
  (('रहे', 'हों

In [49]:
l = list(filter(lambda x: 1 < int(x[1]), ngrams_up_to_20[0].most_common()))
len(l), l

(262,
 [(('है', 'कि'), 17),
  (('के', 'लिए'), 14),
  (('है', 'तो'), 11),
  (('सकते', 'हैं'), 9),
  (('हम', 'उसे'), 8),
  (('कहते', 'हैं।'), 8),
  (('हैं।', 'अगर'), 8),
  (('हैं', 'कि'), 8),
  (('तो', 'हम'), 7),
  (('रहे', 'हैं'), 7),
  (('अपने', 'भीतर'), 6),
  (('हैं', 'तो'), 6),
  (('रहा', 'है'), 6),
  (('के', 'साथ'), 6),
  (('हैं', 'लेकिन'), 6),
  (('कि', 'आप'), 6),
  (('कर', 'रहे'), 6),
  (('रूप', 'से'), 5),
  (('हर', 'चीज'), 5),
  (('जब', 'आप'), 5),
  (('की', 'एक'), 5),
  (('होता', 'है।'), 5),
  (('हैं।', 'आप'), 5),
  (('नहीं', 'है'), 5),
  (('सुखद', 'बन'), 5),
  (('यह', 'है'), 5),
  (('की', 'जरूरत'), 5),
  (('रहे', 'हैं।'), 5),
  (('करने', 'के'), 4),
  (('धरती', 'पर'), 4),
  (('के', 'रूप'), 4),
  (('रूप', 'में'), 4),
  (('तो', 'वह'), 4),
  (('वह', 'बहुत'), 4),
  (('जाता', 'है'), 4),
  (('होते', 'हैं।'), 4),
  (('कर', 'सकते'), 4),
  (('है।', 'अगर'), 4),
  (('बहुत', 'सुखद'), 4),
  (('आनंद', 'या'), 4),
  (('है', 'और'), 4),
  (('तो', 'आप'), 4),
  (('नहीं', 'है।'), 4),
  (('रहे', 'हों'

The `remove_periods` function help us to clean the n-grams. It examines n-grams to determine whether any of the words within them contain characters such as '।', '’', or '‘'. This function is designed to exclude n-grams with these characters when analyzing text. It will return `True` for n-grams that don't contain those characters and `False` for n-grams that do.

In [50]:
def remove_periods(ngram):
    for wrd in ngram[0]:
        if '।' in wrd or "’" in wrd or "‘" in wrd:
            return False
    return True

`my_filter` takes a list of n-grams (groups of words). First, it keeps only the n-grams that appear more than once in the text. Then, it removes any n-grams that contain characters like '।', '’', or '‘'. So, the result is a list of n-grams that meet these two rules.

In [51]:
def my_filter(ngrams):
    return filter(remove_periods, list(filter(lambda x: 1 < int(x[1]), ngrams)))

These `X_train_example` and `y_train_example` lists represent training data for the language model, where each element in these lists contains the index of a word in the vocabulary.

In [52]:
bigrams_to_learn = ngrams_up_to_20[0]
X_train_example = [[word_to_index[sent[0][0]]] for sent in my_filter(bigrams_to_learn.most_common())
                  if sent[0][0] in word_to_index and sent[0][1] in word_to_index]
y_train_example = [[word_to_index[sent[0][1]]] for sent in my_filter(bigrams_to_learn.most_common())
                  if sent[0][0] in word_to_index and sent[0][1] in word_to_index]

In [53]:
X_train_example[0:10], y_train_example[0:10]

([[44], [27], [44], [160], [180], [31], [87], [261], [1], [31]],
 [[199], [14], [87], [31], [244], [199], [180], [31], [2], [87]])

In [54]:
len(X_train_example), len(y_train_example)

(237, 237)

In [55]:
trigrams_to_learn = ngrams_up_to_20[1].copy()
[sent[0] for sent in my_filter(trigrams_to_learn.most_common())]

[('तो', 'हम', 'उसे'),
 ('है', 'तो', 'हम'),
 ('यह', 'है', 'कि'),
 ('के', 'रूप', 'में'),
 ('बहुत', 'सुखद', 'बन'),
 ('है', 'कि', 'आप'),
 ('इस', 'धरती', 'पर'),
 ('कर', 'सकते', 'हैं'),
 ('करने', 'के', 'लिए'),
 ('अगर', 'वह', 'बहुत'),
 ('वह', 'बहुत', 'सुखद'),
 ('सुखद', 'बन', 'जाता'),
 ('बन', 'जाता', 'है'),
 ('जाता', 'है', 'तो'),
 ('के', 'लिए', 'एक'),
 ('कर', 'रहे', 'हों'),
 ('सकते', 'हैं', 'लेकिन'),
 ('की', 'कोशिश', 'कर'),
 ('कोशिश', 'कर', 'रहे'),
 ('सोचते', 'हैं', 'कि'),
 ('डाल', 'तक', 'पहुंच'),
 ('बाहर', 'निकलने', 'का'),
 ('निकलने', 'का', 'रास्ता'),
 ('लेकिन', 'जब', 'आप'),
 ('एक', 'इंसान', 'के'),
 ('को', 'ठीक', 'करने'),
 ('ऐसा', 'क्यों', 'है'),
 ('नहीं', 'है', 'कि'),
 ('नहीं', 'किया', 'है'),
 ('के', 'साथ', 'और'),
 ('क्या', 'है', 'एक'),
 ('प्रसन्नता', 'की', 'एक'),
 ('सुखद', 'बन', 'जाती'),
 ('खोज', 'रहे', 'हैं'),
 ('है', 'तो', 'उसे'),
 ('हैं', 'तो', 'आप'),
 ('तो', 'आप', 'क्या'),
 ('मुख्य', 'रूप', 'से'),
 ('आप', 'चौबीस', 'घंटे'),
 ('भी', 'नहीं', 'है'),
 ('शांति', 'या', 'आनंद'),
 ('की', 'जरूरत'

This code adds more training data to the existing X_train_example and y_train_example lists. The new training data is based on the trigrams in the trigrams_to_learn list. The code extracts sequences of word indices from these trigrams for both input and output.

The my_filter function is used to select and list the trigrams that meet specific conditions.

In [56]:
X_train_example.extend([[word_to_index[w] for w in sent[0][:-1]] for sent in my_filter(trigrams_to_learn.most_common())
               if all([w in word_to_index for w in sent[0]])])
y_train_example.extend([[word_to_index[w] for w in sent[0][1:]] for sent in my_filter(trigrams_to_learn.most_common())
               if all([w in word_to_index for w in sent[0]])])

In [57]:
len(X_train_example), len(y_train_example)

(313, 313)

In [58]:
X_train_example[1575:1585], y_train_example[1575:1585]

([], [])

This code prepares training data using pairs of words (called bigrams) from the ngrams_up_to_20[0] list. Once the training data is created, the code mixes up the order of the elements in the X_train_2 and y_train_2 lists using the Fisher-Yates shuffle.

In [59]:
bigrams_to_learn = ngrams_up_to_20[0]
X_train_2 = [[word_to_index[sent[0][0]]] for sent in my_filter(bigrams_to_learn.most_common())
                  if sent[0][0] in word_to_index and sent[0][1] in word_to_index]
y_train_2 = [[word_to_index[sent[0][1]]] for sent in my_filter(bigrams_to_learn.most_common())
                  if sent[0][0] in word_to_index and sent[0][1] in word_to_index]
X_train_2, y_train_2 = fisher_yates(X_train_2, y_train_2)

In [60]:
len(X_train_2), len(y_train_2)

(237, 237)

In [61]:
X_train_2[0:10], y_train_2[0:10]

([[389], [239], [22], [164], [163], [59], [195], [249], [186], [246]],
 [[18], [424], [18], [61], [81], [25], [31], [451], [599], [31]])

In [62]:
len(X_train_2), len(y_train_2)

(237, 237)

In [63]:
X_train_2[0:10], y_train_2[0:10]

([[389], [239], [22], [164], [163], [59], [195], [249], [186], [246]],
 [[18], [424], [18], [61], [81], [25], [31], [451], [599], [31]])

In [64]:
X_train[0:10], y_train[0:10], len(X_train), len(y_train)

([[0], [0], [0], [0], [0], [0], [0], [0], [0], [0]],
 [[332], [61], [78], [78], [147], [163], [183], [81], [647], [509]],
 127,
 127)

In [65]:
X_train.extend(X_train_2)
y_train.extend(y_train_2)

In [66]:
len(X_train), len(y_train)

(364, 364)

In [67]:
random.sample(list(zip(X_train, y_train)), 10)

[([26], [27]),
 ([91], [44]),
 ([206], [44]),
 ([303], [31]),
 ([193], [514]),
 ([0], [42]),
 ([0], [37]),
 ([0], [23]),
 ([199], [65]),
 ([0], [92])]

In [68]:
ngrams_to_learn = ngrams_up_to_20[1]
ngrams_to_learn.most_common(10)

[(('तो', 'हम', 'उसे'), 7),
 (('है', 'तो', 'हम'), 6),
 (('कहते', 'हैं।', 'अगर'), 6),
 (('यह', 'है', 'कि'), 5),
 (('के', 'रूप', 'में'), 4),
 (('बहुत', 'सुखद', 'बन'), 4),
 (('है', 'कि', 'आप'), 4),
 (('इस', 'धरती', 'पर'), 3),
 (('कर', 'सकते', 'हैं'), 3),
 (('करने', 'के', 'लिए'), 3)]

In [69]:
[sent[0] for sent in my_filter(ngrams_to_learn.most_common(10))]

[('तो', 'हम', 'उसे'),
 ('है', 'तो', 'हम'),
 ('यह', 'है', 'कि'),
 ('के', 'रूप', 'में'),
 ('बहुत', 'सुखद', 'बन'),
 ('है', 'कि', 'आप'),
 ('इस', 'धरती', 'पर'),
 ('कर', 'सकते', 'हैं'),
 ('करने', 'के', 'लिए')]

In [70]:
[sent[0][:-1] for sent in my_filter(ngrams_to_learn.most_common(10))]

[('तो', 'हम'),
 ('है', 'तो'),
 ('यह', 'है'),
 ('के', 'रूप'),
 ('बहुत', 'सुखद'),
 ('है', 'कि'),
 ('इस', 'धरती'),
 ('कर', 'सकते'),
 ('करने', 'के')]

In [71]:
[sent[0][1:] for sent in my_filter(ngrams_to_learn.most_common(10))]

[('हम', 'उसे'),
 ('तो', 'हम'),
 ('है', 'कि'),
 ('रूप', 'में'),
 ('सुखद', 'बन'),
 ('कि', 'आप'),
 ('धरती', 'पर'),
 ('सकते', 'हैं'),
 ('के', 'लिए')]

In [72]:
[[word_to_index[w] for w in sent[0]] for sent in my_filter(ngrams_to_learn.most_common(10))
    if all([w in word_to_index for w in sent[0]])]

[[87, 180, 244],
 [44, 87, 180],
 [61, 44, 199],
 [27, 22, 18],
 [89, 241, 247],
 [44, 199, 65],
 [16, 79, 80],
 [166, 160, 31],
 [26, 27, 14]]

In [73]:
X_train_2 = [[word_to_index[w] for w in sent[0][:-1]] for sent in my_filter(ngrams_to_learn.most_common(10))
    if all([w in word_to_index for w in sent[0]])]
X_train_2

[[87, 180],
 [44, 87],
 [61, 44],
 [27, 22],
 [89, 241],
 [44, 199],
 [16, 79],
 [166, 160],
 [26, 27]]

The code further modifies the `y_train_2` variable to contain the indices of words in the second position of a trigram for a subset of the most common trigrams.

In [74]:
y_train_2 = [[word_to_index[w] for w in sent[0][1:]] for sent in my_filter(ngrams_to_learn.most_common(10))
    if all([w in word_to_index for w in sent[0]])]
y_train_2

[[180, 244],
 [87, 180],
 [44, 199],
 [22, 18],
 [241, 247],
 [199, 65],
 [79, 80],
 [160, 31],
 [27, 14]]

In [75]:
X_train_2 = [[word_to_index[w] for w in sent[0][:-1]] for sent in my_filter(ngrams_to_learn.most_common())
               if all([w in word_to_index for w in sent[0]])]
y_train_2 = [[word_to_index[w] for w in sent[0][1:]] for sent in my_filter(ngrams_to_learn.most_common())
               if all([w in word_to_index for w in sent[0]])]
X_train_2, y_train_2 = fisher_yates(X_train_2, y_train_2)
X_train_2[0:5], y_train_2[0:5], len(X_train_2), len(y_train_2)

([[16, 79], [27, 22], [241, 247], [193, 415], [8, 309]],
 [[79, 80], [22, 18], [247, 122], [415, 59], [309, 143]],
 76,
 76)

In [76]:
def my_filter(ngrams):
    return filter(remove_periods, ngrams)

We made several changes to the X_train_2 and y_train_2 training data. We limited the number of examples to 2000 and mixed up the order of the examples using the Fisher-Yates shuffle. Finally, we printed the first 5 elements and the lengths of both lists.

In [77]:
X_train_2 = [[word_to_index[w] for w in sent[0][:-1]] for sent in my_filter(ngrams_to_learn.most_common())
               if all([w in word_to_index for w in sent[0]])]
y_train_2 = [[word_to_index[w] for w in sent[0][1:]] for sent in my_filter(ngrams_to_learn.most_common())
               if all([w in word_to_index for w in sent[0]])]
X_train_2 = X_train_2[:2000]
y_train_2 = y_train_2[:2000]
X_train_2, y_train_2 = fisher_yates(X_train_2, y_train_2)
X_train_2[0:5], y_train_2[0:5], len(X_train_2), len(y_train_2)

([[651, 44], [259, 458], [42, 82], [65, 143], [448, 18]],
 [[44, 324], [458, 649], [82, 393], [143, 469], [18, 174]],
 1902,
 1902)

In [78]:
ngrams_to_learn = ngrams_up_to_20[1]
X_train_2 = [[word_to_index[w] for w in sent[0][:-1]] for sent in my_filter(ngrams_to_learn.most_common())
               if all([w in word_to_index for w in sent[0]])]
y_train_2 = [[word_to_index[w] for w in sent[0][1:]] for sent in my_filter(ngrams_to_learn.most_common())
               if all([w in word_to_index for w in sent[0]])]
print(X_train_2[0:5], y_train_2[0:5], len(X_train_2), len(y_train_2))

[[87, 180], [44, 87], [61, 44], [27, 22], [89, 241]] [[180, 244], [87, 180], [44, 199], [22, 18], [241, 247]] 1902 1902


In [79]:
word_to_index['END']

33

This code finds and counts groups of three words (called trigrams) that come at the end of sentences. It does this by looking for the "END" token in the output data.

In [80]:
def check_eos(trigram):
    if trigram[1] == word_to_index['END']:
          return True
    return False

trigrams_eos = list(filter(check_eos, y_train_2))
len(trigrams_eos), trigrams_eos[0:5]

(0, [])

### Adding sets of n-grams to the training data 
This code gathers and prepares training data from multiple groups of words (called n-grams) and adds it to the existing training data. This allows us to train the model on a larger dataset. The tqdm library shows a progress bar so we can track how long the code is taking to run.

In [81]:
from tqdm import tqdm
for i in tqdm(range(1, len(ngrams_up_to_20))):
    ngrams_to_learn = ngrams_up_to_20[i]
    X_train_2 = [[word_to_index[w] for w in sent[0][:-1]] for sent in my_filter(ngrams_to_learn.most_common())
                   if all([w in word_to_index for w in sent[0]])]
    y_train_2 = [[word_to_index[w] for w in sent[0][1:]] for sent in my_filter(ngrams_to_learn.most_common())
                   if all([w in word_to_index for w in sent[0]])]
    X_train_2 = X_train_2[:2000]
    y_train_2 = y_train_2[:2000]
    X_train_2, y_train_2 = fisher_yates(X_train_2, y_train_2)
    X_train.extend(X_train_2)
    y_train.extend(y_train_2)

100%|██████████| 18/18 [00:00<00:00, 83.68it/s] 


This code is using the `random.sample` function to randomly sample 100 pairs of elements from the `X_train` and `y_train` lists, which are zipped together.

In [82]:
print(random.sample(list(zip(X_train, y_train)), 100))

[([658, 193], [193, 659]), ([585, 8, 586, 8, 587, 389, 18, 459, 380, 160, 31, 65, 509, 27, 14, 81, 393], [8, 586, 8, 587, 389, 18, 459, 380, 160, 31, 65, 509, 27, 14, 81, 393, 314]), ([8, 164, 282, 283, 284, 56, 285, 27, 279, 280, 65, 286, 287, 27, 14, 288], [164, 282, 283, 284, 56, 285, 27, 279, 280, 65, 286, 287, 27, 14, 288, 236]), ([214, 138, 160, 31, 215, 216, 157, 13, 165, 143, 52, 44], [138, 160, 31, 215, 216, 157, 13, 165, 143, 52, 44, 180]), ([180, 197, 22, 13, 61, 143, 198, 160, 199, 180, 1, 200, 13, 194, 201, 194], [197, 22, 13, 61, 143, 198, 160, 199, 180, 1, 200, 13, 194, 201, 194, 202]), ([165, 143, 166, 206, 44], [143, 166, 206, 44, 87]), ([351, 14, 41, 332, 462, 80], [14, 41, 332, 462, 80, 544]), ([412, 44, 88, 42, 66, 413, 44, 34, 1, 414, 193, 415, 59, 2, 27, 416], [44, 88, 42, 66, 413, 44, 34, 1, 414, 193, 415, 59, 2, 27, 416, 262]), ([236, 2, 263, 44, 87, 244, 251, 257, 56], [2, 263, 44, 87, 244, 251, 257, 56, 70]), ([671, 672, 193, 140, 673, 674, 527, 42, 675, 676, 

In [83]:
len(X_train), len(y_train)

(20503, 20503)

In [84]:
len(tokenized_conversation)

127

In [85]:
tokenized_conversation[65]

['START',
 'मानवीय',
 'प्रणाली',
 'धरती',
 'पर',
 'सबसे',
 'परिष्कृत',
 'भौतिक',
 'रूप',
 'है',
 '।',
 'END']

In [86]:
[[word_to_index[w] for w in sent] for sent in tokenized_conversation if all([w in word_to_index for w in sent])][51]

[0,
 87,
 78,
 264,
 390,
 42,
 391,
 91,
 199,
 92,
 75,
 382,
 392,
 393,
 8,
 394,
 395,
 87,
 65,
 207,
 396,
 257,
 56,
 397,
 236,
 56,
 278,
 398,
 399,
 400,
 44,
 32,
 33]

In [87]:
X_train_full_sentences = [[word_to_index[w] for w in sent[:-1]] for sent in tokenized_conversation
                         if all([w in word_to_index for w in sent])]
y_train_full_sentences = [[word_to_index[w] for w in sent[1:]] for sent in tokenized_conversation
                         if all([w in word_to_index for w in sent])]

In [88]:
print(X_train_full_sentences[0:5], y_train_full_sentences[0:5])

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 13, 23, 24, 25, 26, 27, 5, 28, 13, 29, 30, 31, 32], [0, 19, 34, 35, 36, 18, 37, 38, 39, 40, 41, 42, 40, 8, 43, 18, 8, 44, 32], [0, 45, 34, 46, 47, 48, 49, 50, 51, 52, 49, 53, 54, 55, 56, 57, 58, 59, 60, 49, 61, 62, 34, 63, 52, 64, 65, 41, 42, 66, 67, 40, 68, 69, 70, 32], [0, 71, 72, 18, 73, 74, 73, 75, 76, 77, 32], [0, 78, 65, 16, 79, 80, 81, 82, 83, 27, 22, 18, 84, 85, 86, 87, 88, 89, 90, 91, 32]] [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 13, 23, 24, 25, 26, 27, 5, 28, 13, 29, 30, 31, 32, 33], [19, 34, 35, 36, 18, 37, 38, 39, 40, 41, 42, 40, 8, 43, 18, 8, 44, 32, 33], [45, 34, 46, 47, 48, 49, 50, 51, 52, 49, 53, 54, 55, 56, 57, 58, 59, 60, 49, 61, 62, 34, 63, 52, 64, 65, 41, 42, 66, 67, 40, 68, 69, 70, 32, 33], [71, 72, 18, 73, 74, 73, 75, 76, 77, 32, 33], [78, 65, 16, 79, 80, 81, 82, 83, 27, 22, 18, 84, 85, 86, 87, 88, 89, 90, 91, 32, 33]]


In [89]:
print(tokenized_conversation[51], tokenized_conversation[51][::-1])

['START', 'तो', 'अगर', 'आपके', 'पास', 'एक', 'विकल्प', 'होता', 'कि', 'आपकी', 'जीवन', 'ऊर्जा', 'किस', 'तरह', 'की', 'अभिव्यक्ति', 'पाए', 'तो', 'आप', 'क्या', 'चुनते', 'आनंद', 'या', 'क्लेश', 'प्रसन्नता', 'या', 'अप्रसन्नता', 'इसका', 'उत्तर', 'स्पष्ट', 'है', '।', 'END'] ['END', '।', 'है', 'स्पष्ट', 'उत्तर', 'इसका', 'अप्रसन्नता', 'या', 'प्रसन्नता', 'क्लेश', 'या', 'आनंद', 'चुनते', 'क्या', 'आप', 'तो', 'पाए', 'अभिव्यक्ति', 'की', 'तरह', 'किस', 'ऊर्जा', 'जीवन', 'आपकी', 'कि', 'होता', 'विकल्प', 'एक', 'पास', 'आपके', 'अगर', 'तो', 'START']


In [90]:
len(tokenized_conversation)

127

In [91]:
print(random.sample(tokenized_conversation, 20))

[['START', 'इंसानइंसान', 'के', 'बीच', 'तरीके', 'बदल', 'सकते', 'हैं', 'लेकिन', 'चाहे', 'आप', 'पैसा', 'कमाने', 'की', 'कोशिश', 'कर', 'रहे', 'हों', 'शराब', 'पी', 'रहे', 'हों', 'या', 'स्वर्ग', 'जाने', 'की', 'कोशिश', 'कर', 'रहे', 'हों', 'खुशी', 'ही', 'एकमात्र', 'लक्ष्य', 'है', '।', 'END'], ['START', 'अपने', 'भीतर', 'मुड़ना', 'ही', 'एकमात्र', 'समाधान', 'हैसद्गुरु', 'की', 'नवीनतम', 'पुस्तक', '‘इनर', 'इंजीनरिंग’', 'से', 'लिए', 'गए', 'इस', 'अंश', 'में', 'सद्गुरु', 'हमें', 'स्थायी', 'रूप', 'से', 'खुशहाली', 'का', 'अनुभव', 'करने', 'के', 'एकमात्र', 'तरीके', 'से', 'परिचित', 'कराते', 'हैं', '।', 'END'], ['START', 'खेत', 'के', 'किनारे', 'पर', 'एक', 'बड़े', 'पेड़', 'को', 'देखकर', 'तीतर', 'ने', 'कहा', '‘आह', 'एक', 'समय', 'था', 'जब', 'मैं', 'उस', 'पेड़', 'की', 'सबसे', 'ऊंची', 'डाल', 'तक', 'उड़', 'सकता', 'था', '।', 'END'], ['START', 'यह', 'कुछ', 'ऐसा', 'है', '।', 'END'], ['START', 'आप', 'घर', 'से', 'निकले', 'बिना', 'भी', 'यह', 'काम', 'कर', 'सकते', 'हैं', 'मानव', 'इतिहास', 'में', 'इससे', 'पहले', 'यह', 'कभी', 'सं

This code works with a dataset of sentences that have been broken into words. It generates variations of sentences by taking the last i words of a sentence (where i can be from 3 to 19). The code then randomly selects 65 sentences from the dataset and takes the last i words of each sentence. Finally, it prints 10 of these sequences at random.

In [92]:
import random
last_n_words = []
for i in range(3, 20):
    tokenized_sentences_400 = random.sample(list(tokenized_conversation), 65)
    for s in tokenized_sentences_400:
        last_n_words.append(s[::-1][:i][::-1])

print(random.sample(last_n_words, 10))

[['START', 'दो', 'सप्ताह', 'में', 'ही', 'तुम', 'सबसे', 'ऊपर', 'पहुंच', 'जाओगे', '।', 'END'], ['में', 'रहेगा', '।', 'END'], ['पर', 'गौर', 'कीजिएः', 'जिस', 'पल', 'उनका', 'पेट', 'भर', 'जाता', 'है', 'वे', 'काफी', 'शांतिमय', 'होते', 'हैं', '।', 'END'], ['भीतर', 'अनुभव', 'किया', 'है', '।', 'END'], ['होगा', '।', 'END'], ['है', '।', 'END'], ['कहते', 'हैं', '।', 'END'], ['START', 'स्टॉक', 'मार्केट', 'इसे', 'चकनाचूर', 'कर', 'सकता', 'है', '।', 'END'], ['होते', 'हैं', '।', 'END'], ['तरीका', 'नहीं', 'है', '।', 'END']]


In [93]:
len(last_n_words)

1105

In [94]:
X_train_eos = [[word_to_index[w] for w in sent[:-1]] for sent in last_n_words
                         if all([w in word_to_index for w in sent])]
y_train_eos = [[word_to_index[w] for w in sent[1:]] for sent in last_n_words
                         if all([w in word_to_index for w in sent])]

In [95]:
len(X_train_eos), len(y_train_eos)

(1105, 1105)

In [96]:
print(X_train_eos[0:10], y_train_eos[0:10] )

[[44, 32], [31, 32], [44, 32], [44, 32], [31, 32], [603, 32], [77, 32], [144, 32], [31, 32], [425, 32]] [[32, 33], [32, 33], [32, 33], [32, 33], [32, 33], [32, 33], [32, 33], [32, 33], [32, 33], [32, 33]]


In [97]:
print(X_train_eos[1000:1010], y_train_eos[1000:1010])

[[81, 393, 314, 23, 588, 160, 31, 71, 124, 61, 44, 199, 61, 589, 143, 44, 32], [27, 693, 694, 257, 27, 613, 80, 16, 695, 696, 59, 697, 267, 268, 77, 44, 32], [0, 116, 236, 2, 263, 44, 87, 244, 251, 257, 56, 70, 246, 31, 32], [0, 539, 89, 561, 85, 514, 527, 545, 99, 51, 52, 32], [22, 13, 1, 336, 39, 81, 27, 219, 193, 39, 40, 27, 219, 337, 86, 31, 32], [44, 42, 608, 609, 610, 18, 42, 89, 90, 611, 4, 262, 606, 24, 607, 44, 32], [0, 97, 98, 99, 100, 101, 87, 88, 42, 89, 102, 103, 104, 32], [0, 65, 42, 154, 155, 35, 156, 157, 8, 158, 8, 120, 159, 160, 31, 32], [0, 78, 239, 249, 241, 250, 44, 87, 180, 244, 251, 246, 31, 32], [0, 78, 239, 240, 241, 242, 243, 44, 87, 180, 244, 245, 246, 31, 32]] [[393, 314, 23, 588, 160, 31, 71, 124, 61, 44, 199, 61, 589, 143, 44, 32, 33], [693, 694, 257, 27, 613, 80, 16, 695, 696, 59, 697, 267, 268, 77, 44, 32, 33], [116, 236, 2, 263, 44, 87, 244, 251, 257, 56, 70, 246, 31, 32, 33], [539, 89, 561, 85, 514, 527, 545, 99, 51, 52, 32, 33], [13, 1, 336, 39, 81, 2

In [98]:
len(X_train), len(y_train)

(20503, 20503)

In [99]:
X_train.extend(X_train_eos)
y_train.extend(y_train_eos)

In [100]:
len(X_train), len(y_train)

(21608, 21608)

In [101]:
len(X_train)

21608

In [102]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Determine the length of the longest sequence
max_seq_length = max(len(sequence) for sequence in X_train)

# Pad all sequences to the length of the longest one
X_train_padded = pad_sequences(X_train, maxlen=max_seq_length, padding='post')
y_train_padded = pad_sequences(y_train, maxlen=max_seq_length, padding='post')

# Now you can safely convert to NumPy arrays
X_train2 = np.array(X_train_padded)
y_train2 = np.array(y_train_padded)


In [103]:
X_train2.shape, y_train2.shape

((21608, 19), (21608, 19))

In [104]:
print(random.sample(list(zip(X_train2, y_train2)), 10))

[(array([156, 103,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0], dtype=int32), array([103,  98,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0], dtype=int32)), (array([240, 193, 249,  27, 355,  24, 220,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0], dtype=int32), array([193, 249,  27, 355,  24, 220, 339,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0], dtype=int32)), (array([165, 143, 166, 206,  44,  87, 207,  61, 208,  24, 209, 143,   0,
         0,   0,   0,   0,   0,   0], dtype=int32), array([143, 166, 206,  44,  87, 207,  61, 208,  24, 209, 143,  44,   0,
         0,   0,   0,   0,   0,   0], dtype=int32)), (array([244, 252,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0], dtype=int32), array([252, 246,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0], dtype=int32)),

In [105]:
vocabulary_size

720

In [109]:
#We only need t perform this step once.
# pip install gensim

In [110]:
vocabulary_size

720

In [111]:
len(most_common_words)

720

This code loads a pre-trained FastText word embedding model and extracts word vectors from it. The embeddings_index dictionary can be used to look up word vectors for specific words later on. You can download the pre-trained FastText model from https://fasttext.cc/docs/en/english-vectors.html

In [112]:
import os
import gzip
import shutil
import requests

# Specify the URL for the FastText word vectors
fasttext_url = 'https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.hi.300.vec.gz'

# Specify the file paths for downloading and unzipping
download_path = 'cc.hi.300.vec.gz'
unzipped_path = 'cc.hi.300.vec'

# Download the FastText word vectors file
response = requests.get(fasttext_url, stream=True)
with open(download_path, 'wb') as file:
    for chunk in response.iter_content(chunk_size=1024):
        if chunk:
            file.write(chunk)

# Unzip the downloaded file
with gzip.open(download_path, 'rb') as f_in, open(unzipped_path, 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)

# Clean up: Remove the gzipped file
os.remove(download_path)

print(f'FastText word vectors downloaded and unzipped to: {unzipped_path}')


FastText word vectors downloaded and unzipped to: cc.hi.300.vec


In [113]:
import os
from gensim.models import KeyedVectors

# Path to the downloaded FastText model file
fasttext_model_path = r"cc.hi.300.vec"

# Initialize the FastText model
fasttext_model = KeyedVectors.load_word2vec_format(fasttext_model_path)

# Initialize a dictionary to store word vectors
embeddings_index = {}

# Iterate through the words in the FastText model
for word in fasttext_model.index_to_key:
    embeddings_index[word] = fasttext_model[word]

print('Found %s word vectors.' % len(embeddings_index))

Found 1876653 word vectors.


In [114]:
embedding_dim = 300

embedding_matrix = np.zeros((vocabulary_size, embedding_dim))
for word, i in most_common_words:
    embedding_vector = embeddings_index.get(word)
    if i < vocabulary_size:
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

The embedding_matrix variable contains vectors that represent the words in our vocabulary. These vectors were created using the pre-trained FastText model.

In [115]:
embedding_matrix.shape

(720, 300)

In [116]:
most_common_words[200]

('खोजा', 2)

In [117]:
embedding_matrix[12]

array([-3.39000002e-02, -6.72999993e-02,  3.89999989e-03,  6.01999983e-02,
       -1.31000001e-02, -3.80000006e-03,  4.76000011e-02, -1.97999999e-02,
       -6.58999979e-02,  1.75000001e-02,  5.42000011e-02, -2.06000004e-02,
        9.20000020e-03,  2.30000000e-02,  3.15999985e-02, -8.39999970e-03,
        1.21500000e-01, -3.80000006e-03,  2.19999999e-03,  7.98000023e-02,
        1.89999994e-02, -9.30000003e-03,  1.77999996e-02, -6.00000005e-03,
        2.16000006e-02,  3.33999991e-02, -1.18000004e-02, -2.38000005e-02,
        2.83000004e-02, -8.99999961e-03, -2.00000009e-03, -2.91000009e-02,
        4.80999984e-02,  8.60000029e-03,  1.43099993e-01,  3.40999998e-02,
        3.80000006e-03, -7.59999976e-02,  6.66000023e-02, -5.53999990e-02,
       -3.48000005e-02,  2.43999995e-02, -3.10999993e-02, -3.48000005e-02,
        1.54999997e-02,  3.64999995e-02,  1.27999997e-02,  4.94999997e-02,
        3.29000019e-02, -1.17700003e-01,  7.10000005e-03, -2.38000005e-02,
        1.57999992e-02,  

The find_closest_embeddings function takes a word vector as input and returns a list of words from our vocabulary, sorted by how similar they are to the input word vector. The similarity is calculated using the Euclidean distance between the vectors. This function can be used to find words in our vocabulary that are semantically similar to the input word, based on how their vectors are represented.

In [118]:
from scipy import spatial

def find_closest_embeddings(embedding):
    return sorted(embeddings_index.keys(), key=lambda word: spatial.distance.euclidean(embeddings_index[word], embedding))

In [119]:
find_closest_embeddings(embeddings_index["मानवता"])[1:6]

['दानवता', 'हैमानवता', 'विश्वमानवता', 'अमानवता', 'महामानवता']

In [120]:
print(find_closest_embeddings(
    embeddings_index["पास"] - embeddings_index["एक"] + embeddings_index["विकल्प"]
)[:10])

['पास', 'विकल्प', 'समानो', 'अपीले', 'सैंडबॉक्स', 'लगेतो', 'लगताएक', 'राष्ट््रीय', 'समूचित', 'डिपोर्ट']


This code uses a technique called t-Distributed Stochastic Neighbor Embedding (t-SNE) to reduce the number of dimensions in the data. This helps us to understand how the data points are related to each other.

In [121]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=3, random_state=0)

In [122]:
words =  list(embeddings_index.keys())[:500]
vectors = [embeddings_index[word] for word in words]

In [123]:
# Only need to perform this step once
# pip install plotly

In [124]:
import plotly.express as px

In [125]:
vocabulary_size, embedding_dim

(720, 300)

In [126]:
y_train2[1000]

array([150, 133,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0], dtype=int32)

### building, compiling, and training a recurrent neural network (RNN) 

In [127]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

vocab_size = 1030
embedding_dim = 300
max_length = 19

# Define the RNN model architecture
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    SimpleRNN(units=500, return_sequences=True),
    Dense(vocab_size, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train2, y_train2, batch_size=128, epochs=20)

Epoch 1/20
169/169 [==============================] - 103s 583ms/step - loss: 2.7381 - accuracy: 0.5727
Epoch 2/20
169/169 [==============================] - 76s 451ms/step - loss: 1.0630 - accuracy: 0.8091
Epoch 3/20
169/169 [==============================] - 88s 524ms/step - loss: 0.2781 - accuracy: 0.9450
Epoch 4/20
169/169 [==============================] - 82s 487ms/step - loss: 0.1740 - accuracy: 0.9569
Epoch 5/20
169/169 [==============================] - 71s 421ms/step - loss: 0.1440 - accuracy: 0.9593
Epoch 6/20
169/169 [==============================] - 79s 467ms/step - loss: 0.1311 - accuracy: 0.9599
Epoch 7/20
169/169 [==============================] - 74s 437ms/step - loss: 0.1246 - accuracy: 0.9602
Epoch 8/20
169/169 [==============================] - 71s 420ms/step - loss: 0.1209 - accuracy: 0.9600
Epoch 9/20
169/169 [==============================] - 74s 438ms/step - loss: 0.1183 - accuracy: 0.9602
Epoch 10/20
169/169 [==============================] - 75s 441ms/step - 

In [128]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 19, 300)           309000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 19, 500)           400500    
                                                                 
 dense (Dense)               (None, 19, 1030)          516030    
                                                                 
Total params: 1225530 (4.68 MB)
Trainable params: 1225530 (4.68 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Generating sentences using a trained language model. It uses a pre-trained model to predict the next words in a sequence, sampling from the model's output to create coherent sentences.

In [129]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_sentence_tf(model, sentence_start_token, sentence_end_token, unknown_token, word_to_index, index_to_word, senten_max_length):
    new_sentence = [word_to_index[sentence_start_token]]
    start_token_index = word_to_index[sentence_start_token]

    while not new_sentence[-1] == word_to_index[sentence_end_token] and len(new_sentence) < senten_max_length:
        padded_sentence = pad_sequences([new_sentence], maxlen=19, padding='post')
        next_word_probs = model.predict(padded_sentence)[0]

        # Set the probability of the start token to 0 for this and all subsequent predictions
        next_word_probs[:, start_token_index] = 0

        # Renormalize probabilities to sum to 1 after modifying the 'start' token probability
        next_word_probs /= np.sum(next_word_probs, axis=1, keepdims=True)

        # Sample a word from the probability distribution
        sampled_word = np.random.choice(range(len(next_word_probs[-1])), p=next_word_probs[-1])

        # Avoid sampling unknown or start token by setting their probabilities to 0 and renormalizing
        while sampled_word in [word_to_index[unknown_token], start_token_index]:
            next_word_probs[-1][sampled_word] = 0
            next_word_probs[-1] /= np.sum(next_word_probs[-1])
            sampled_word = np.random.choice(range(len(next_word_probs[-1])), p=next_word_probs[-1])

        new_sentence.append(sampled_word)

    sentence_str = [index_to_word[x] for x in new_sentence[1:-1]]  # Exclude the start token
    return sentence_str


num_sentences = 10
senten_min_length = 7
senten_max_length = 20

for i in range(num_sentences):
    sent = generate_sentence_tf(model, sentence_start_token, sentence_end_token, unknown_token, word_to_index, index_to_word, senten_max_length)
    if len(sent) >= senten_min_length:
        print(" ".join(sent))

1/1 [==============================] - 0s 59ms/step
उसने सपने का हम होती किसी । तो । एक अनुभव क्या पैदा भीतर अपने तो पूरा कहानी
1/1 [==============================] - 0s 92ms/step
हर है अपने किया अपने है लेकिन हैं अपने मानव सफलता तो से वे आपको सकते तो अपने
1/1 [==============================] - 0s 48ms/step
से का उसने चाहे और जाती बदलने को मानवता तो भीतर तो से कम एक एक स्थितियों किसी
1/1 [==============================] - 0s 58ms/step
रहा है रहा है फिर लेकिन एक देखो पूरी से पूरी अपने अपने वे सफलता कि असल हम
1/1 [==============================] - 0s 40ms/step
खाना पूरा से पखवाड़े आधार वे हैं कम उसने पूरी का तो तो लेकिन लिए जाती सबसे वह
1/1 [==============================] - 0s 46ms/step
गया वे गया का तो खुद अपने अपने हैं तो है प्रणाली का रहा है प्रणाली अपने से
1/1 [==============================] - 0s 45ms/step
वे अपने आज पूरी असल एक को आपकी मैं किसी आपकी पूरी मानवता चाहे अपने कौन पूरी किसे
1/1 [==============================] - 0s 43ms/step


1/1 [==============================] - 0s 39ms/step
तो वैसा सफलता आपको जाती कर आज वे से अभी । भीतर वह है सप्ताह कमाए
1/1 [==============================] - 0s 42ms/step
तो हम और वे भीतर एक अपने अपने यह किसी के भीतर अपने हम और एक पूरी सद्गुरु
